In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


examples = [
{
"movie": "매드맥스",
"answer": """
🔥🚗💨
""",
},
{
"movie": "서울의 봄",
"answer": """
🚨🪖🔥
""",
},
]

llm  = ChatOpenAI(model_name="gpt-4o",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationBufferMemory(
    return_messages=True,
)

example_prompt = ChatPromptTemplate.from_messages([
        ("human","영화 {movie}을 이모지 3개로 표현해줘"),
        ("ai", "{answer}")
    ]
)

few_shot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You know every movie. If a human tells you the title of the movie, you have to respond with 3 emoticons.",
        ),
        few_shot_example_prompt,
        (
            "system",
            "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]
movie_chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie):
    result = movie_chain.invoke({
        "movie": movie,
    })
    memory.save_context({"input": movie},{"output": result.content}) 
    return result

In [6]:
invoke_chain("Dune")


AIMessage(content='🌌🏜️🔮', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f7a584cf1f'}, id='run-0e70a865-7920-4fc0-be60-f4a20c5f9688-0')

In [7]:
invoke_chain("oldboy")


AIMessage(content='🔒🔨😮', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f7a584cf1f'}, id='run-53fa1eeb-c579-4b14-8aa1-776999a78a01-0')

In [8]:
invoke_chain("Tell me the movie that i asked to you before in english not emoji")


AIMessage(content='The movie you asked about before is "Oldboy."', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f7a584cf1f'}, id='run-7687c9c1-cc26-4b07-80cc-694bb5151878-0')